# Kaggle OCRデータセットの前処理

このNotebookでは、Kaggle OCRデータセットからYOLO形式のラベルファイルを生成します。

## 処理の流れ
1. 設定のインポート
2. データの確認と分析
3. ラベルファイルの生成（重い処理）
4. 統計情報の確認

In [ ]:
# 必要なライブラリのインポート
import json
import sys
from pathlib import Path
from collections import defaultdict
from typing import Dict, List
from tqdm import tqdm
import pandas as pd

# 設定ファイルからクラス定義をインポート
sys.path.insert(0, str(Path.cwd() / 'config'))
from classes import CLASSES, CLASS_TO_ID, latex_to_class

print(f"対象クラス数: {len(CLASSES)}")
print(f"クラス一覧: {CLASSES}")

In [ ]:
# パスの設定（必要に応じて変更してください）
BASE_DIR = Path.cwd()
SOURCE_DIR = BASE_DIR / 'raw' / 'aidapearson_ocr'
TARGET_DIR = BASE_DIR / 'processed'
SPLIT = 'train'  # 'train', 'val', 'test'

# 出力ディレクトリを作成
labels_dir = TARGET_DIR / SPLIT / 'labels'
labels_dir.mkdir(parents=True, exist_ok=True)

print(f"ソースディレクトリ: {SOURCE_DIR}")
print(f"出力ディレクトリ: {labels_dir}")
print(f"分割: {SPLIT}")

# JSONファイルを検索
json_files = list(SOURCE_DIR.rglob('kaggle_data_*.json'))
print(f"\n見つかったJSONファイル: {len(json_files)} 件")
for i, json_file in enumerate(json_files[:5], 1):
    print(f"  {i}. {json_file.name}")
if len(json_files) > 5:
    print(f"  ... 他 {len(json_files) - 5} 件")

In [ ]:
# ヘルパー関数の定義

def convert_bbox_to_yolo_format(
    xmin: float,
    ymin: float,
    xmax: float,
    ymax: float
) -> tuple[float, float, float, float]:
    """バウンディングボックスをYOLO形式に変換"""
    center_x = (xmin + xmax) / 2.0
    center_y = (ymin + ymax) / 2.0
    width = xmax - xmin
    height = ymax - ymin
    
    # 0.0-1.0の範囲にクリップ
    center_x = max(0.0, min(1.0, center_x))
    center_y = max(0.0, min(1.0, center_y))
    width = max(0.0, min(1.0, width))
    height = max(0.0, min(1.0, height))
    
    return center_x, center_y, width, height


def extract_labels_from_sample(sample: Dict) -> List[Dict]:
    """サンプルから正解ラベルとbboxを抽出"""
    if 'image_data' not in sample:
        return []
    
    img_data = sample['image_data']
    visible_chars = img_data.get('visible_latex_chars', [])
    
    xmins = img_data.get('xmins', [])
    xmaxs = img_data.get('xmaxs', [])
    ymins = img_data.get('ymins', [])
    ymaxs = img_data.get('ymaxs', [])
    
    results = []
    
    for i, latex_char in enumerate(visible_chars):
        # LaTeX文字をクラスに変換
        class_char = latex_to_class(latex_char)
        
        if class_char is None:
            continue
        
        if i >= len(xmins) or i >= len(xmaxs) or i >= len(ymins) or i >= len(ymaxs):
            continue
        
        class_id = CLASS_TO_ID[class_char]
        xmin = xmins[i]
        ymin = ymins[i]
        xmax = xmaxs[i]
        ymax = ymaxs[i]
        
        center_x, center_y, width, height = convert_bbox_to_yolo_format(
            xmin, ymin, xmax, ymax
        )
        
        results.append({
            'class': class_char,
            'class_id': class_id,
            'bbox_yolo': (center_x, center_y, width, height)
        })
    
    return results

print("ヘルパー関数を定義しました")

## データの確認（オプション）

最初のJSONファイルを読み込んで、データ構造を確認します。

In [ ]:
# 最初のJSONファイルを読み込んで確認（オプション）
if len(json_files) > 0:
    sample_json = json_files[0]
    print(f"サンプルファイル: {sample_json.name}")
    
    with open(sample_json, 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    
    print(f"サンプル数: {len(sample_data)}")
    
    # 最初のサンプルを確認
    if len(sample_data) > 0:
        first_sample = sample_data[0]
        print(f"\n最初のサンプル:")
        print(f"  UUID: {first_sample.get('uuid', 'N/A')}")
        print(f"  LaTeX: {first_sample.get('latex', 'N/A')[:100]}...")
        
        if 'image_data' in first_sample:
            img_data = first_sample['image_data']
            print(f"  画像サイズ: {img_data.get('width', 'N/A')} x {img_data.get('height', 'N/A')}")
            visible_chars = img_data.get('visible_latex_chars', [])
            print(f"  可視文字数: {len(visible_chars)}")
            
            # ラベルを抽出してみる
            labels = extract_labels_from_sample(first_sample)
            print(f"  抽出されたラベル数: {len(labels)}")
            if len(labels) > 0:
                print(f"  例: {labels[0]}")
else:
    print("JSONファイルが見つかりません")

## ラベルファイルの生成（重い処理）

このセルは時間がかかります。すべてのJSONファイルを処理してラベルファイルを生成します。

In [ ]:
# ラベルファイルの生成（重い処理）
stats = defaultdict(int)
total_processed = 0
total_skipped = 0
total_samples = 0

# 各JSONファイルを処理
for json_path in tqdm(json_files, desc="JSONファイル処理中"):
    # JSONファイルを読み込む
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    total_samples += len(data)
    
    # 各サンプルを処理
    for sample in tqdm(data, desc=f"  {json_path.name}", leave=False):
        uuid = sample.get('uuid', '')
        if not uuid:
            total_skipped += 1
            continue
        
        # ラベルを抽出
        labels = extract_labels_from_sample(sample)
        
        if len(labels) == 0:
            total_skipped += 1
            continue
        
        # ラベルファイルを生成
        label_path = labels_dir / f"{uuid}.txt"
        with open(label_path, 'w', encoding='utf-8') as f:
            for label in labels:
                class_id = label['class_id']
                center_x, center_y, width, height = label['bbox_yolo']
                
                # YOLO形式で書き込み
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
                
                # 統計情報を更新
                stats[label['class']] += 1
        
        total_processed += 1

print("\n" + "=" * 80)
print("処理完了")
print("=" * 80)
print(f"総サンプル数: {total_samples}")
print(f"処理済み: {total_processed} サンプル")
print(f"スキップ: {total_skipped} サンプル（{len(CLASSES)}クラスに該当する文字がない）")
print(f"生成されたラベルファイル: {total_processed} 件")

## 統計情報の表示

クラス別のインスタンス数を確認します。

In [ ]:
# クラス別統計を表示
print("クラス別統計:")
print("-" * 60)
for cls in CLASSES:
    count = stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")

total_instances = sum(stats.values())
print("-" * 60)
print(f"総インスタンス数: {total_instances}")
print(f"\n出力先: {labels_dir}")

In [ ]:
# 統計情報をDataFrameで表示（見やすくするため）
df_stats = pd.DataFrame([
    {'クラス': cls, 'インスタンス数': stats[cls], '割合 (%)': f"{stats[cls]/total_instances*100:.2f}" if total_instances > 0 else "0.00"}
    for cls in CLASSES
])
df_stats = df_stats.sort_values('インスタンス数', ascending=False)

print("クラス別統計（降順）:")
print(df_stats.to_string(index=False))

## 生成されたラベルファイルの確認

生成されたラベルファイルの例を確認します。

In [ ]:
# 生成されたラベルファイルの例を表示
label_files = list(labels_dir.glob('*.txt'))
if len(label_files) > 0:
    print(f"生成されたラベルファイル数: {len(label_files)}")
    print(f"\n最初のラベルファイルの例:")
    print(f"ファイル名: {label_files[0].name}")
    print("\n内容:")
    with open(label_files[0], 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines[:10], 1):  # 最初の10行
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                class_name = CLASSES[class_id] if class_id < len(CLASSES) else 'unknown'
                print(f"  {i}. クラス: {class_name} (ID: {class_id}), "
                      f"bbox: ({parts[1]}, {parts[2]}, {parts[3]}, {parts[4]})")
        if len(lines) > 10:
            print(f"  ... 他 {len(lines) - 10} 行")
else:
    print("ラベルファイルが見つかりません")